### Building a Better (and More Realistic) Momentum Strategy
Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:
* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
    - Suppose A stock show consistant perform even small (0.5% or 1%) consider hqm 
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.
    - A stock which shows sudden or some month higher return (e.g. 10%) and remaning month lower 
----
The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by `short-term news` that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).
To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentile
* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation hqm often. It stands for high-quality-momentum stocks.



In [214]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf 
import talib
import datetime as dt
from scipy.stats import percentileofscore as score
from dataclasses import dataclass

In [213]:
@dataclass
class HQM:
    df: pd.DataFrame
    portfolio_value: float = 100000
    required_col = [
        'One Year Return', 
        'Six Month Return', 
        'Three Month Return', 
        'One Month Return'
    ]

    def _get_null_value(self) -> pd.DataFrame:
        """Return the count of null values in each column."""
        return self.df.isnull().sum()

    def get_stock_stats(self) -> pd.DataFrame:
        try:
            null_values = self._get_null_value()
            if null_values.sum() == 0:
                position_size = self.portfolio_value / len(self.df)
                number_of_share_to_buy = (position_size // self.df.iloc[-1]).astype('int64')

                changes = {}
                periods = {
                    'year5': 252 * 5,
                    'year3': 252 * 3,
                    'year2': 252 * 2,
                    'year1': 252,
                    'month6': 21 * 6,
                    'month3': 21 * 3,
                    'month1': 21,
                    'day30': 30,
                    'day5': 5
                }

                max_value = self.df.max()
                min_value = self.df.min()
                latest_price = self.df.iloc[-1]
                changes['maxChanges'] = ((max_value - min_value) / min_value) * 100

                for period_name, period_number in periods.items():
                    if period_number >= len(self.df):
                        continue
                    past_price = self.df.iloc[-period_number]
                    changes[f"{period_name}ChangePercentage"] = (latest_price - past_price) / past_price * 100

                dataframe = pd.DataFrame.from_dict(changes, orient='index').T
                dataframe['latest Price'] = latest_price
                dataframe['Number of Shares to buy'] = number_of_share_to_buy

                return dataframe
            else:
                raise ValueError(f'Null values found in: {null_values[null_values > 0]}')
        except Exception as e:
            self.get_exception(e)

    def get_momentum_data(self) -> pd.DataFrame:
        try:
            df = self.get_stock_stats()
            if df is None:
                raise ValueError("Failed to get stock stats, cannot proceed.")

            new_df = pd.DataFrame()
            new_df_col = ['latest Price', 'Number of Shares to buy', 'year1ChangePercentage', 'month6ChangePercentage', 'month3ChangePercentage', 'month1ChangePercentage']

            for col in new_df_col:
                if col in df.columns:
                    new_df[col] = df[col]
                    if col not in ['latest Price', 'Number of Shares to buy']:
                        new_df[f'{col}Percentile'] = df[col].rank(pct=True)
                else:
                    new_df[col] = np.nan

            update_col = ['latest Price', 'Number of Shares to buy', 'One Year Return', 'One Year Return Percentile', 'Six Month Return', 
                          'Six Month Return Percentile', 'Three Month Return', 'Three Month Return Percentile', 
                          'One Month Return', 'One Month Return Percentile']

            new_df.columns = update_col

            for col in self.required_col:
                changed_col = col 
                percentile_col = f'{col} Percentile'
                new_df[percentile_col] = new_df[changed_col].rank(pct=True)

            return new_df
        except Exception as e:
            self.get_exception(e)

    def get_hqm_score(self) -> pd.DataFrame:
        try:
            df = self.get_momentum_data()
            if df is None:
                raise ValueError("Failed to get momentum data, cannot proceed.")

            df['HQM Score'] = df[[f'{col} Percentile' for col in self.required_col]].mean(axis=1)
            return df
        except Exception as e:
            self.get_exception(e)

    def get_exception(self, exception):
        print(f'An error occurred: {exception}')


# Example usage
ticker_list =  ['AAPL', 'MSFT', 'GOOG', 'AMZN', 'TSLA', 'JPM', 'BAC', 'V', 'MA', 'PG', 'KO', 'WMT', 'PEP', 'JNJ', 'UNH', 'PFE', 'LLY']
stock_df = yf.download(ticker_list, start='2020-01-01', end=dt.date.today(), interval='1D')['Adj Close']
hqm = HQM(stock_df)
hqm_score_df = hqm.get_hqm_score()
print(hqm_score_df)


[*********************100%%**********************]  17 of 17 completed


,latest Price,Number of Shares to buy,One Year Return,One Year Return Percentile,Six Month Return,Six Month Return Percentile,Three Month Return,Three Month Return Percentile,One Month Return,One Month Return Percentile,HQM Score
AAPL,208.139999,0,12.087729,0.411765,7.195544,0.411765,21.991395,1.000000,11.376281,1.000000,0.705882
AMZN,185.570007,0,43.485660,0.764706,20.625333,0.764706,3.260809,0.411765,2.496550,0.529412,0.617647
BAC,40.020000,2,48.508997,0.941176,22.129623,0.823529,9.233332,0.764706,2.791447,0.588235,0.779412
GOOG,180.789993,0,47.127076,0.882353,27.641552,0.882353,19.745764,0.941176,3.390685,0.647059,0.838235
JNJ,149.119995,0,-7.065342,0.176471,-2.178528,0.176471,-3.153337,0.117647,-0.387443,0.235294,0.176471
JPM,198.880005,0,47.030058,0.823529,20.163349,0.647059,2.679444,0.352941,0.995331,0.352941,0.544118
KO,63.970001,1,7.874347,0.352941,12.057274,0.529412,6.732395,0.705882,3.827295,0.705882,0.573529
LLY,890.109985,0,95.440369,1.000000,56.152106,1.000000,15.325444,0.882353,10.100807,0.941176,0.955882
MA,456.959991,0,21.019009,0.529412,8.235908,0.470588,-3.875112,0.058824,1.274351,0.411765,0.367647
MSFT,447.670013,0,34.670155,0.647059,20.283449,0.705882,6.058153,0.647059,4.840753,0.764706,0.691176


In [212]:
ticker_list =  ['AAPL', 'MSFT', 'GOOG', 'AMZN', 'TSLA', 'JPM', 'BAC', 'V', 'MA', 'PG', 'KO', 'WMT', 'PEP', 'JNJ', 'UNH', 'PFE','LLY']
crypto_list = ['BTC-USD', 'ETH-USD', 'XRP-USD', 'LTC-USD', 'BCH-USD', 'XMR-USD', 'ADA-USD', 'DASH-USD', 'ETC-USD', 'ZEC-USD', 'XLM-USD', 'DOGE-USD']

In [178]:
stock_df = yf.download(ticker_list, start='2020-01-01', end=dt.date.today(), interval='1D')['Adj Close']
stockobj = HQM(stock_df).get_hqm_score()
stockobj

[*********************100%%**********************]  17 of 17 completed


,latest Price,Number of Shares to buy,One Year Return,One Year Return Percentile,Six Month Return,Six Month Return Percentile,Three Month Return,Three Month Return Percentile,One Month Return,One Month Return Percentile,HQM Score
Ticker,,,,,,,,,,,
AAPL,208.139999,0,12.087729,0.411765,7.195544,0.411765,21.991395,1.000000,11.376281,1.000000,0.705882
AMZN,185.570007,0,43.485660,0.764706,20.625333,0.764706,3.260809,0.411765,2.496550,0.529412,0.617647
BAC,40.020000,2,48.508997,0.941176,22.129623,0.823529,9.233332,0.764706,2.791447,0.588235,0.779412
GOOG,180.789993,0,47.127076,0.882353,27.641552,0.882353,19.745764,0.941176,3.390685,0.647059,0.838235
JNJ,149.119995,0,-7.065342,0.176471,-2.178528,0.176471,-3.153337,0.117647,-0.387443,0.235294,0.176471
JPM,198.880005,0,47.030058,0.823529,20.163349,0.647059,2.679444,0.352941,0.995331,0.352941,0.544118
KO,63.970001,1,7.874347,0.352941,12.057274,0.529412,6.732395,0.705882,3.827295,0.705882,0.573529
LLY,890.109985,0,95.440369,1.000000,56.152106,1.000000,15.325444,0.882353,10.100807,0.941176,0.955882
MA,456.959991,0,21.019009,0.529412,8.235908,0.470588,-3.875112,0.058824,1.274351,0.411765,0.367647
